In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf

from PIL import Image
import os
import matplotlib.pyplot as plt

#load images convert to greyscale and resize to 48x48 pixels

target_size = 48
input_dir = "C:\\Users\\ThinkPad\\Desktop\\BME\\4. szemeszter\\Machine learning\\project\\dataset1\\part1" # change to your input directory obv
test_dir = ".\\test"
def load_images(input_dir, target_size):
    
    image_names = os.listdir(input_dir)
    path = os.path.join(input_dir, image_names[0])
    img = Image.open(path).convert("L")

    images = [Image.open(os.path.join(input_dir, name)).convert("L").resize((target_size, target_size), Image.LANCZOS) for name in image_names]

    return image_names, images

def plot_images(img):
    plt.imshow(img)
    plt.axis('off')
    plt.show()

image_names, images = load_images(input_dir, target_size) #this takes a while lol
test_names, test_images = load_images(test_dir, target_size)
from sklearn.preprocessing import OneHotEncoder
#getting labels from filenames

base = np.array([names.split(".")[0] for names in image_names]) # swap image_names here for for part 2 and 3 if needed
parts = np.array([name.split("_")[:3] for name in base])

age = np.array([int(part[0]) for part in parts]) # 0-116 int
gender = np.array([part[1] for part in parts]) # 0 = M, 1 = F
ethnicity = np.array([part[2] for part in parts]) # 0 = White, 1 = Black, 2 = Asian, 3 = Indian, 4 = Other

enc = OneHotEncoder(sparse_output=False)

age_norm = age / 116.0
gender_norm = gender
ethnicity_norm = enc.fit_transform(ethnicity.reshape(-1, 1)) # one hot encode


test_base = np.array([names.split(".")[0] for names in test_names]) # swap image_names here for for part 2 and 3 if needed
test_parts = np.array([name.split("_")[:3] for name in test_base])

test_age = np.array([int(part[0]) for part in test_parts]) # 0-116 int
test_gender = np.array([part[1] for part in test_parts]) # 0 = M, 1 = F
test_ethnicity = np.array([part[2] for part in test_parts]) # 0 = White, 1 = Black, 2 = Asian, 3 = Indian, 4 = Other

enc = OneHotEncoder(sparse_output=False)

test_age_norm = test_age / 116.0
test_gender_norm = test_gender
test_ethnicity_norm = enc.fit_transform(test_ethnicity.reshape(-1, 1)) # one hot encode

# we build the convulutional neural network

cnn = input_data(shape = [None, target_size, target_size, 1]) # input layer
for i in [32, 64, 128, 64, 32]:
    # convulution + activation + pooling
    cnn = conv_2d(cnn, i, 5)
    cnn = max_pool(cnn, 5)

cnn = fully_connected(cnn, 1024, activation='softmax')
cnn = regression(cnn, optimizer ='adam', learning_rate = LR)

model = tflearn.DNN(cnn)
# x: images y: labels
model.fit({'input': images}, {'targets': gender_norm}, n_epoch = 5, 
    validation_set =({'input': test_images}, {'targets': test_gender_norm}), 
    snapshot_step = 500, show_metric = True)



In [ ]:
# we test the model
gender_out =[]
for i in test_images:
    gender_out.append(model.predict(i))[0]

In [ ]:
# d.c. al fine